# the `Dict` type

In [1]:
    from schemata import *
    import typing; raises = __import__("pytest").raises(BaseException)

In [2]:
    assert Dict(foo=1)\
    == Dict({"foo": 1})\
    == dict(foo=1)

## default `Dict`s

In [3]:
    assert Dict.default(foo=42)()\
    == dict(foo=42)

In [4]:
    assert Dict.default(foo=42)(foo=1)\
    == dict(foo=1)

In [5]:
    class DictWDefaults(Dict):
        foo: Integer = 1
        bar: String = "abc"
            
    assert DictWDefaults\
    == Dict[dict(
       foo=Integer.default(1), bar=String.default("abc") 
    )]

    assert DictWDefaults() == dict(foo=1, bar="abc")

when we compose type methods with the value "null" we indicate _to call this function to create foo_. in general, we think it is best practice to return the value of the key in any method sharing a namespace.

In [6]:
    class DictWFunctionDefault(Dict.cast()):
        foo: String
            
        def foo(self: ["null"]):
            return "xyz"

    DictWFunctionDefault.schema(True)

{'cast': True, 'type': 'object', 'properties': {'foo': {'type': 'string'}}}

In [7]:
    DictWFunctionDefault(), DictWFunctionDefault("abc")

({}, {'foo': 'abc'})

## `Dict.Properties`

In [8]:
    assert Dict.properties(foo=Integer)\
    == Dict[dict(foo=Integer)]

In [9]:
    assert Dict[dict(foo=Integer, bar=Float)].py()\
    == typing.Dict[str, typing.Union[float, int]]

## `Dict.Required`

## `Dict.AdditionalProperties`

In [10]:
    assert Dict.additionalProperties(Integer)\
    == Dict[Integer]

### default dictionaries

when `Dict.AdditionalProperties` are defined the dictionary behaves as a python dictionary.

In [11]:
    default_dict_list = Dict[list]()
    default_dict_list["a"].append(1)
    dict(default_dict_list)

{'a': [1]}

## `Dict.PropertyNames`

the `Dict.PropertyNames` type defines the key type of a dictionary. `json` does not allow for non string keys, but python does

In [12]:
    assert Dict.propertyNames(Integer)\
    == Dict[Integer,]
    assert Dict.propertyNames(Integer).additionalProperties(String)\
    == Dict[Integer, String]
    assert Dict[Integer, String].py()\
    == typing.Dict[int, str]

## `Dict.PatternProperties`

## `Dict.Dependencies`

dictionaries with dependencies in their function signature, on the `self` argument of a method function, we can define all of the key on key depends on.

In [13]:
    class DictWDeps(Dict):
        foo: Integer
        bar: String
            
        def bar(self: "foo"):
            return str(self["foo"])
    DictWDeps.schema()

{'type': 'object',
 'properties': {'foo': schemata.numbers.Integer,
  'bar': schemata.strings.String}}

In [14]:
DictWDeps(dict(foo=1, bar='abc'))

{'foo': 1, 'bar': 'abc'}

In [15]:
d = DictWDeps()
d["foo"] = 100
d

{'foo': 100}

In [16]:
    class DictWTypedDeps(Dict):
        foo: Integer            
        bar: String
        def bar(self: Dict.Required["foo",]):
            return str(self["foo"])
    DictWTypedDeps.schema(True)

{'type': 'object',
 'properties': {'foo': {'type': 'integer'}, 'bar': {'type': 'string'}},
 'dependencies': {'bar': {'required': ('foo',)}}}

In [17]:
    DictWTypedDeps(dict(foo=1))
    with raises:
        DictWTypedDeps(dict(foo=1, bar=200))

In [18]:
    class DictWFunctionTypedDeps(Dict):
        def foo(self: "null") -> Integer:
            return self.get("foo", 100)
        def bar(self: Dict.Required["foo",]) -> String:
            return str(self["foo"])
    DictWFunctionTypedDeps.schema(True)

{'type': 'object',
 'properties': {'foo': {'type': 'integer'}, 'bar': {'type': 'string'}},
 'dependencies': {'bar': {'required': ('foo',)}}}

In [19]:
    class ListWFunctionTypedDeps(List):
        def foo(self: "null") -> Integer:
            return self.get("foo", 100)
        def bar(self: Dict.Required["foo",]) -> String:
            return str(self["foo"])
        
    ListWFunctionTypedDeps.schema(True)

{'type': 'array',
 'items': {'properties': {'foo': {'type': 'integer'},
   'bar': {'type': 'string'}},
  'dependencies': {'bar': {'required': ('foo',)}}}}